# Eager Few Shot Object Detection Colab

Welcome to the Eager Few Shot Object Detection Colab --- in this colab we demonstrate fine tuning of a (TF2 friendly) RetinaNet architecture on very few examples of a novel class after initializing from a pre-trained COCO checkpoint.
Training runs in eager mode.

Estimated time to run through this colab (with GPU): < 5 minutes.

## Imports

In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
pip install PyYAML=="5.1"


In [ ]:
pip install pillow=="9.5"

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt


import os
import random
import io
import imageio
import glob
import pandas as pd
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import matplotlib
import matplotlib.pyplot as plt

import cv2
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import PIL.ImageOps
import tensorflow as tf

import os
import numpy as np
import pandas as pd
from os import path
import matplotlib.patches as patches
import matplotlib.pylab as plt
from PIL import Image


from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

# Utilities

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def load_image_into_numpy_array(path,color=1):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata().convert('RGB')).reshape(
      (im_height, im_width,3)).astype(np.uint8)


def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None,
                    min_score_thresh=0.8):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=min_score_thresh)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Cargar datos para entrenamiento.

In [ ]:
def Cargar_imagen(Name,Path):
  img=cv2.imread(os.path.join(Path, Name+".jpg"))
  img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return(img)

In [ ]:
# Graficar(img,pd.DataFrame(Data,columns=["Xmin","Ymin","Xmax","Ymax"]))
def Graficar(img,df,MODEL):

  # Imagen Principal
  plt.clf()
  fig, ax = plt.subplots()
  final_image = Image.fromarray(img)
  # Impresión con escala para mejor visualizacion
  fig.set_size_inches(10, 10, forward=True)
  ax.imshow(final_image, cmap='gray', vmin=0, vmax=255)
  #
  #dibujar puntos de enfermedad
  img_height,img_width=np.array(img).shape[0],np.array(img).shape[1]
  # print(img_height,img_height)
  if (MODEL=="O"):
    for i in df.index:
        # Create a Rectangle patch
          xcenter= int((float(df["Xcenter"][i])*img_width))
          ycenter= int((float(df["Ycenter"][i])*img_height))
          Width= int((float(df["Width"][i])*img_width))
          Height= int((float(df["Height"][i])*img_height))
          xmin = int(xcenter - (Width/2))
          ymin = int(ycenter - (Height/2))
  #     Ver Imagen con Restricciones
          rect = patches.Rectangle((xmin,ymin), int(Width), int(Height), linewidth=2, edgecolor='r', facecolor='none')
          plt.text(xmin, ymin,str(i))
          ax.add_patch(rect)
    plt.savefig('O.png')

  if (MODEL=="Y"):
    for i in df.index:#"Class","Xcenter","Ycenter","Width","Height","Score"
          # Create a Rectangle patch
          xcenter= int((float(df["Xcenter"][i])*img_width))
          ycenter= int((float(df["Ycenter"][i])*img_height))
          Width= int((float(df["Width"][i])*img_width))
          Height= int((float(df["Height"][i])*img_height))
          xmin = int(xcenter - (Width/2))
          ymin = int(ycenter - (Height/2))
          print(xmin,ymin,Width,Height)
    #     Ver Imagen con Restricciones
          rect = patches.Rectangle((xmin,ymin), int(Width), int(Height), linewidth=2, edgecolor='r', facecolor='none')
          plt.text(xmin, ymin,str(i))
          ax.add_patch(rect)
    plt.savefig('Y.png')

  if (MODEL=="R"):
    for i in df.index:
        # Create a Rectangle patch
        xmin= int((float(df["Xmin"][i])*img_width))
        ymin= int((float(df["Ymin"][i])*img_height))
        xmax= int((float(df["Xmax"][i])*img_width))
        ymax= int((float(df["Ymax"][i])*img_height))
        Width = xmax-xmin
        Height= ymax-ymin
        print(xmin,ymin,Width,Height)
  #     Ver Imagen con Restricciones
        rect = patches.Rectangle((xmin,ymin), int(Width), int(Height), linewidth=2, edgecolor='r', facecolor='none')
        plt.text(xmin, ymin,str(i))
        ax.add_patch(rect)
    plt.savefig('R.png')
  plt.show()

In [ ]:
def Generador_data(Name,path,MODEL):
  DF=[]

  with open(path+Name+".txt", 'r') as txt_file:
    df=txt_file.read()
  # print(df)
  df2=df.split("\n")
  # print(df2)
  for i in range(len(df2)):
    df3 = df2[i].split(" ")
    if (df3!=[]):
      DF.append(df3)
  DF.pop(-1)
  # print(DF)
  # print(len(DF))
  if MODEL=="O":
    Data=pd.DataFrame(DF,columns=["Class","Xcenter","Ycenter","Width","Height"])
  if MODEL=="Y":
    Data=pd.DataFrame(DF,columns=["Class","Xcenter","Ycenter","Width","Height","Score"])
  if MODEL=="R":
    Data=pd.DataFrame(DF,columns=["Class","Ymin","Xmin","Ymax","Xmax","Score"])
  Data = Data.drop(["Class"], axis=1)
  return(Data)


In [ ]:
BASE_FOLDER="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/"
Lista_imagenes = os.listdir(BASE_FOLDER+'images/')
Lista_imagenes.sort()
print(len(Lista_imagenes))

In [ ]:
#Cargar Los datos
for i in range(0,1):#len(Lista_imagenes)):

  Name = Lista_imagenes[i]
  Name =  Name.split(".")
  print(Name)
  print("Original")
  Path="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/"
  img = Cargar_imagen(Name[0],Path+"images")
  Data=Generador_data(Name[0],Path+"labels/","O")
  Graficar(img,Data,"O")


  #cargar Datos Yolo
  print("YOLO")
  Path="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/predYOLO/"
  Data=Generador_data(Name[0],Path,"Y")
  Graficar(img,Data,"Y")

  #Cargar DAtos Resnet
  print("PresResNet50")
  Path="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/predResNet50/"
  Data=Generador_data(Name[0],Path,"R")
  Graficar(img,Data,"R")



# **Parametrizacion de datos finales**

In [ ]:
BASE_FOLDER="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/"
Lista_imagenes = os.listdir(BASE_FOLDER+'images/')
Lista_imagenes.sort()
print(len(Lista_imagenes))

In [ ]:
import numpy as np

def calculate_iou(box1, box2):
    """
    Calcula el IoU (Intersección sobre Unión) entre dos bounding boxes.

    Parameters:
        box1 (list): [x1, y1, x2, y2] de la primera bounding box.
        box2 (list): [x1, y1, x2, y2] de la segunda bounding box.

    Returns:
        float: El valor de IoU.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return float(iou)

def calculate_ap(precision, recall):
    """
    Calcula el Average Precision (AP) a partir de las curvas de precision y recall.

    Parameters:
        precision (list): Lista de valores de precision.
        recall (list): Lista de valores de recall.

    Returns:
        float: El valor de AP.
    """
    m_recall = np.concatenate(([0.0], recall, [1.0]))
    m_precision = np.concatenate(([0.0], precision, [0.0]))

    for i in range(len(m_precision) - 1, 0, -1):
        m_precision[i - 1] = max(m_precision[i - 1], m_precision[i])

    indices = np.where(m_recall[1:] != m_recall[:-1])[0] + 1
    ap = np.sum((m_recall[indices] - m_recall[indices - 1]) * m_precision[indices])
    return ap

def calculate_map(ground_truth, predictions, iou_threshold=0.2):
    """
    Calcula el Mean Average Precision (mAP) entre bounding boxes.

    Parameters:
        ground_truth (list): Lista de bounding boxes de etiquetas verdaderas.
        predictions (list): Lista de bounding boxes de predicciones del modelo.
        iou_threshold (float): Umbral de IoU para considerar una detección como verdadera positiva.

    Returns:
        float: El valor de mAP.
    """
    num_images = len(ground_truth)
    ap_values = []
    List_Mediciones=[]
    for i in range(num_images):
        gt_boxes = ground_truth[i]
        pred_boxes = predictions[i]

        num_gt_boxes = len(gt_boxes)
        num_pred_boxes = len(pred_boxes)

        if num_pred_boxes == 0:
            ap_values.append(0)
            continue

        iou_matrix = np.zeros((num_pred_boxes, num_gt_boxes))

        for j in range(num_pred_boxes):
            for k in range(num_gt_boxes):
                iou_matrix[j][k] = calculate_iou(pred_boxes[j], gt_boxes[k])

        true_positive = np.zeros(num_pred_boxes)
        false_positive = np.zeros(num_pred_boxes)

        for j in range(num_pred_boxes):
            matched_gt_box = -1
            max_iou = -1

            for k in range(num_gt_boxes):
                if iou_matrix[j][k] > max_iou:
                    max_iou = iou_matrix[j][k]
                    matched_gt_box = k

            if max_iou >= iou_threshold and matched_gt_box >= 0:
                true_positive[j] = 1
                iou_matrix[:, matched_gt_box] = -1  # Mark the matched ground truth box as used
            else:
                false_positive[j] = 1

        cumulative_true_positive = sum(true_positive)
        print(true_positive)
        cumulative_false_positive = sum(false_positive)
        print(false_positive)
        precision = cumulative_true_positive / (cumulative_true_positive + cumulative_false_positive)
        recall = cumulative_true_positive / num_gt_boxes

        ap = calculate_ap(precision, recall)
        List_Mediciones.append([i,sum(true_positive),sum(false_positive),num_gt_boxes,precision,recall,ap])
        ap_values.append(ap)

    mAP = np.mean(ap_values,)
    return (mAP,List_Mediciones)



In [ ]:
def Conversor(img,df,MODEL):
  img_height,img_width=np.array(img).shape[0],np.array(img).shape[1]
  Aux=[]
  if MODEL=="O" or MODEL=="Y":
    for i in df.index:
      # Create a Rectangle patch
        xcenter= int((float(df["Xcenter"][i])*img_width))
        ycenter= int((float(df["Ycenter"][i])*img_height))
        Width= int((float(df["Width"][i])*img_width))
        Height= int((float(df["Height"][i])*img_height))
        xmin = int(xcenter - (Width/2))
        ymin = int(ycenter - (Height/2))
        xmax = int(xcenter + (Width/2))
        ymax = int(ycenter + (Height/2))
        Aux.append([xmin,ymin,xmax,ymax])
  if MODEL=="R":
    for i in df.index:
      # Create a Rectangle patch
        xmin= int((float(df["Xmin"][i])*img_width))
        ymin= int((float(df["Ymin"][i])*img_height))
        xmax= int((float(df["Xmax"][i])*img_width))
        ymax= int((float(df["Ymax"][i])*img_height))
        Aux.append([xmin,ymin,xmax,ymax])
  return(Aux)

In [ ]:
from functools import total_ordering
# PArametrizacion final de datos
ground_truth_boxes=[]
predicted_boxes = []
predicted_boxes2 = []

for i in range(0,len(Lista_imagenes)):

  Name = Lista_imagenes[i]
  Name =  Name.split(".")
  print(Name)
  Path="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/"
  img = Cargar_imagen(Name[0],Path+"images")
  DataO=Generador_data(Name[0],Path+"labels/","O")
  ground_truth_boxes.append(Conversor(img,DataO,"O"))
  # display(ground_truth_boxes)
  #cargar Datos Yolo
  Path="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/predYOLO/"
  DataY=Generador_data(Name[0],Path,"Y")
  predicted_boxes.append(Conversor(img,DataY,"Y"))
  # display(predicted_boxes)
  #Cargar DAtos Resnet
  Path="/content/drive/MyDrive/Trabajo_final_VIU/YOLOV8/Dataset/test/predResNet50/"
  DataR=Generador_data(Name[0],Path,"R")
  predicted_boxes2.append(Conversor(img,DataR,"R"))
  # display(predicted_boxes2)
mAP,List_MedicionesYOLO = calculate_map(ground_truth_boxes, predicted_boxes)
print("Mean Average Precision YOLO(mAP):", mAP)
mAP2,List_MedicionesRN50 = calculate_map(ground_truth_boxes, predicted_boxes2)
print("Mean Average Precision ResNet50(mAP):", mAP2)

In [ ]:
display(List_MedicionesYOLO)

In [ ]:
import pandas as pd
df_MedicionesYOLO = pd.DataFrame(List_MedicionesYOLO,columns=["Num_img","TP","FP","GROUNT","Prec","Recall","ap"])
save_path = path.join('/content/drive/MyDrive/Trabajo_final_VIU/Resources/Annotations','df_MedicionesYOLO.csv')
pd.DataFrame(df_MedicionesYOLO).to_csv(save_path)

In [ ]:
display(List_MedicionesRN50)

In [ ]:
import pandas as pd
df_MedicionesRN50 = pd.DataFrame(List_MedicionesRN50,columns=["Num_img","TP","FP","GROUNT","Prec","Recall","ap"])
save_path = path.join('/content/drive/MyDrive/Trabajo_final_VIU/Resources/Annotations','df_MedicionesRN50.csv')
pd.DataFrame(df_MedicionesRN50).to_csv(save_path)

In [ ]:

display(Conversor(img,DataO,"O"))

In [ ]:
display(ground_truth_boxes[0])
display(predicted_boxes[0])
calculate_iou(ground_truth_boxes[0][0], predicted_boxes[0][0])
calculate_iou(ground_truth_boxes[0][1], predicted_boxes[0][1])